In [7]:
import torch
from torch import nn
import requests
import zipfile
from pathlib import Path

In [8]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [9]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3080'

In [10]:

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "soil_low_med_high"

In [12]:
import os
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [13]:
walk_through_dir(image_path)

There are 2 directories and 0 images in 'data\soil_low_med_high'.
There are 3 directories and 0 images in 'data\soil_low_med_high\test'.
There are 0 directories and 0 images in 'data\soil_low_med_high\test\high'.
There are 0 directories and 0 images in 'data\soil_low_med_high\test\low'.
There are 0 directories and 0 images in 'data\soil_low_med_high\test\medium'.
There are 3 directories and 0 images in 'data\soil_low_med_high\train'.
There are 0 directories and 0 images in 'data\soil_low_med_high\train\high'.
There are 0 directories and 0 images in 'data\soil_low_med_high\train\low'.
There are 0 directories and 0 images in 'data\soil_low_med_high\train\medium'.
